# remove folder

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name=version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [1]:
# import importlib, sys, subprocess
# packages = "numpy, matplotlib, astropy, astroquery, version_information" # required modules
# pkgs = packages.split(", ")
# for pkg in pkgs :
#     if not importlib.util.find_spec(pkg):
#         #print(f"**** module {pkg} is not installed")
#         subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
#     else: 
#         print(f"**** module {pkg} is installed")

# %load_ext version_information
# import time
# now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
# print(f"This notebook was generated at {now} ")

# vv = %version_information {packages}
# for i, pkg in enumerate(vv.packages):
#     print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

### import modules

In [2]:
from glob import glob
from pathlib import Path
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.stats import sigma_clip
from ccdproc import combine, ccd_process, CCDData
from pathlib import Path
import ysfitsutilpy as yfu
import ysphotutilpy as ypu
from astropy.io import fits
import _astro_utilities
import _Python_utilities

plt.rcParams.update({'figure.max_open_warning': 0})

In [3]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C2-Asteroid"
TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C3-EXO"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    MASTERDIR = Path(BDFDIR[0]) / _astro_utilities.master_dir
    if not MASTERDIR.exists():
        os.makedirs("{}".format(str(MASTERDIR)))
        print("{} is created...".format(str(MASTERDIR)))
    print ("MASTERDIR: ", format(MASTERDIR))
except : 
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = 'Kepler-17b_LIGHT_-_2024-06-26_-_RiLA600_STX-16803_-_2bin'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/-_CAL-BDF_-_2023_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/120LACHESIS_LIGHT_-_2023-10-09_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/120LACHESIS_LIGHT_-_2023-10-16_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/120LACHESIS_LIGHT_-_2023-11-14_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/1266TONE_LIGHT_-_2023-11-21_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/127JOHANNA_LIGHT_-_2023-11-17_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF

In [184]:
DOINGDIR = Path(DOINGDIRs[0])


In [185]:
DOINGDIR.parent / DOINGDIR.stem

PosixPath('/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin/HAT-P-37b_LIGHT_-_2024-06-28_-_RiLA600_STX-16803_-_2bin')

In [186]:
# DOINGDIR = Path(DOINGDIRs[0])
# print("DOINGDIR", DOINGDIR)

# shutil.move(f"{DOINGDIR/"REDUC_nightsky"}", f"{DOINGDIR/_astro_utilities.reduced_nightsky_dir}")
# print(f"{DOINGDIR/"REDUC_nightsky"} is move to {DOINGDIR/_astro_utilities.reduced_nightsky_dir} ...")
# # shutil.rmtree(DOINGDIR/_astro_utilities.master_dir)
# # print(f"{DOINGDIR/_astro_utilities.master_dir} is removed...")


In [5]:
for DOINGDIR in DOINGDIRs[:] :
    DOINGDIR = Path(DOINGDIR)
    print("DOINGDIR", DOINGDIR)
    # print((DOINGDIR / "REDUC_nightsky"))
    # print((DOINGDIR / "REDUC_nightsky").exists())
    rm_folders = ["reduced", "reduced_nightsky", "reduced_DPhot_Mag12.5", "reduced_DPhot_Mag12.5_fw4",
                  "reduced_nightsky_DPhot_Mag12.5", "reduced_nightsky_DPhot_Mag12.5_fw4"]
    for rm_folder in rm_folders : 
        if (DOINGDIR/rm_folder).exists() :
            shutil.rmtree(DOINGDIR/rm_folder)
    


DOINGDIR /mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/120LACHESIS_LIGHT_-_2023-10-09_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/120LACHESIS_LIGHT_-_2023-10-10_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/120LACHESIS_LIGHT_-_2023-10-16_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/120LACHESIS_LIGHT_-_2023-11-14_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/1266TONE_LIGHT_-_2023-11-21_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/127JOHANNA_LIGHT_-_2023-11-17_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/ASTRO_data/C2-Asteroid/-_-_-_2023-_-_GSON300_STF-8300M_-_1bin/127JOHANNA_LIGHT_-_2023-11-24_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/ASTRO

In [188]:
# for DOINGDIR in DOINGDIRs[:] :
#     DOINGDIR = Path(DOINGDIR)
#     print("DOINGDIR", DOINGDIR)
#     removing_lst = [
#                     # _astro_utilities.master_dir,
#                     # _astro_utilities.reduced_dir,
#                     # _astro_utilities.reduced_nightsky_dir,
#                     # _astro_utilities.Asteroid_result_dir,
#                     # _astro_utilities.Diff_Phot_dir,
#                     "reduced_DPhot_Mag12.5",
#                     "reduced_nightsky_DPhot_Mag12.5",
#                     ]
#     for dir_name in removing_lst :
#         if (DOINGDIR/dir_name).exists() :
#             print(f"{DOINGDIR/dir_name} is removed...")
#             shutil.rmtree(str(DOINGDIR/dir_name), ignore_errors=True)

In [189]:
# import shutil
# DOINGDIR = Path(DOINGDIRs[0])
# print("DOINGDIR", DOINGDIR)

# # shutil.rmtree(DOINGDIR/_astro_utilities.master_dir)
# # print(f"{DOINGDIR/_astro_utilities.master_dir} is removed...")
# shutil.rmtree(DOINGDIR/_astro_utilities.master_dir)
# print(f"{DOINGDIR/_astro_utilities.master_dir} is removed...")
